In [1]:
import argparse
import json
import os
import re
import random
import numpy as np
from tqdm import tqdm
import observation_adapter.observation_adapter as observation_adapter

tree_jsons_paths = [
    "/mnt/c/Users/Sean/Downloads/html_to_trees/train_0/results.json",
    "/mnt/c/Users/Sean/Downloads/html_to_trees/train_1/results.json",
    "/mnt/c/Users/Sean/Downloads/html_to_trees/train_2/results.json",
    "/mnt/c/Users/Sean/Downloads/html_to_trees/train_3/results.json",
    "/mnt/c/Users/Sean/Downloads/html_to_trees/train_4/results.json",
    "/mnt/c/Users/Sean/Downloads/html_to_trees/train_5/results.json",
    "/mnt/c/Users/Sean/Downloads/html_to_trees/train_6/results.json",
    "/mnt/c/Users/Sean/Downloads/html_to_trees/train_7/results.json",
    "/mnt/c/Users/Sean/Downloads/html_to_trees/train_8/results.json",
    "/mnt/c/Users/Sean/Downloads/html_to_trees/train_9/results.json",
    "/mnt/c/Users/Sean/Downloads/html_to_trees/train_10/results.json",
]


def clean_high_level_plan(step):
    return step.replace("[", "").replace("]", "").replace("->", "")


def format_high_level_plan(high_level_plan, step_idx):
    result = "; ".join(high_level_plan[step_idx:step_idx + 3])
    return result

def format_prev_actions(high_level_plan, step_idx):
    result = "; ".join(high_level_plan[0:step_idx])
    return result


def observation_template(tree, objective, prev_actions):
    if prev_actions == "":
        prev_actions = "None"
    template = f"""OBSERVATION:\n{tree}\nOBJECTIVE: {objective}\nPREVIOUS ACTIONS: {prev_actions}"""
    return template

def solution_template(action, objective, high_level_plan):
    template = f"""The high level plan for the next 3 steps: {high_level_plan}. In summary, the next action I will perform is {action}."""
    return template


def get_solution_idx(tree, solution_id):
    tree_arr = tree.split("\n")
    solution_idx = -1
    for idx, line in enumerate(tree_arr):
        if line.strip().startswith(f"[{solution_id}]"):
            solution_idx = idx
            break

    return solution_idx


def crop_tree(tree, solution_id, total_lines_to_keep=100):
    tree_arr = tree.split("\n")
    solution_idx = get_solution_idx(tree, solution_id)
    # random number between 0 and total_lines_to_keep // 2
    random_offset = random.randint(0, total_lines_to_keep // 2)
    start_idx = max(0, solution_idx - random_offset)
    end_idx = min(len(tree_arr), solution_idx + total_lines_to_keep - random_offset)

    if end_idx - start_idx < total_lines_to_keep:
        start_idx = max(0, end_idx - total_lines_to_keep)

    return "\n".join(tree_arr[start_idx:end_idx])


def generate_action_str(operation, solution_id, args):
    if operation == "CLICK":
        return f"```click [{solution_id}]```"
    elif operation == "TYPE":
        return f"```type [{solution_id}] [{args}]```"
    elif operation == "HOVER":
        return f"```hover [{solution_id}]```"
    else:
        raise ValueError(f"Unknown operation {operation}")


def fix_type_solution_not_textbox(solution_id, tree, operation):
    if not operation == "TYPE":
        return solution_id

    tree_arr = tree.split("\n")
    solution_idx = get_solution_idx(tree, solution_id)
    for i in range(solution_idx, len(tree_arr)):
        if "StaticText" not in tree_arr[i]:
            return re.findall(r"\[\s*(\d+)\]", tree_arr[i])[0]


results = []
for tree_json_path in tqdm(tree_jsons_paths):
    with open(tree_json_path, "r") as f:
        tree_json = json.load(f)

    for task_idx, task in enumerate(tree_json):
        # if task["task_domain"] != "Shopping":
        #     continue
        tree = task["tree"]
        objective = task["task_objective"]
        solution_id = task["solution_id"]
        high_level_plan = task["task_high_level_plan"]
        step_idx = task["action_idx"]
        operation = task["action_operation"]["original_op"].replace("SELECT", "CLICK")
        operation_args = task["action_operation"]["value"]
        if tree is None:
            print("Skipping tree")
            continue
        if not operation in ("TYPE", "CLICK", "HOVER"):
            # print("Ignoring operation", operation)
            continue
        solution_id = fix_type_solution_not_textbox(solution_id, tree, operation)
        tree = crop_tree(tree, solution_id)
        new_tree, reverse_fn, id_map = observation_adapter.better_tree(tree)
        # print("New tree:", new_tree)
        new_solution_id = None
        for new_id, old_id in id_map.items():
            if old_id == solution_id:
                new_solution_id = new_id
                break
        if new_solution_id is None:
            print("Error, solution id not clickable")
            continue
        action_str = generate_action_str(operation, new_solution_id, operation_args)
        high_level_plan_str = format_high_level_plan(high_level_plan, step_idx)
        # step_str = clean_high_level_plan(high_level_plan[step_idx])
        prev_actions = format_prev_actions(high_level_plan, step_idx)
        input_str = observation_template(new_tree, objective, prev_actions)
        output_str = solution_template(action_str, objective, high_level_plan_str)
        results.append({"input": input_str, "output": output_str})

        if len(results) == 1:
            print(input_str, "\n", output_str)

print("Number of examples:", len(results))
with open("/mnt/c/Users/Sean/Downloads/mind2web_1105.json", "w") as f:
    json.dump(results, f, indent=4)

  9%|▉         | 1/11 [00:05<00:52,  5.25s/it]

OBSERVATION:

Level 1: [0] button 'Profile image'
Level 2: [1] img 'Profile image'
Level 2: [2] StaticText 'DELICIOUS'
Level 2: [3] StaticText 'STARTS'
Level 2: [4] StaticText 'HERE.'
Level 2: [5] StaticText 'Reservation type'
Level 2: [6] combobox '' hasPopup: menu expanded: False
Level 3: [7] menuitem 'Dine in' selected: True
Level 3: [8] menuitem 'Pickup' selected: False
Level 3: [9] menuitem 'Delivery' selected: False
Level 3: [10] menuitem 'Events' selected: False
Level 3: [11] menuitem 'Wineries' selected: False
Level 3: [12] menuitem 'Everything' selected: False
Level 2: [13] StaticText 'Location'
Level 2: [14] textbox '' required: False
Level 3: [15] StaticText 'Columbus, OH'
Level 2: [16] StaticText 'Date'
Level 2: [17] textbox '' required: False
Level 3: [18] StaticText 'Thu, Mar 16'
Level 2: [19] StaticText 'Time'
Level 2: [20] combobox '' hasPopup: menu expanded: False
Level 3: [21] menuitem 'Now' selected: True
Level 3: [22] menuitem '8:30 PM' selected: False
Level 3: [23]

 82%|████████▏ | 9/11 [00:21<00:04,  2.00s/it]

Error, solution id not clickable


100%|██████████| 11/11 [00:26<00:00,  2.38s/it]


Number of examples: 6914
